### Interfaces and Abstract Base Classes

This notebook contains example code from [*Fluent Python*](http://shop.oreilly.com/product/0636920032519.do), by Luciano Ramalho.

Code by Luciano Ramalho, modified by Allen Downey.

MIT License: https://opensource.org/licenses/MIT

**Let's first toggle on doctest mode**

In [4]:
%doctest_mode

Exception reporting mode: Plain
Doctest mode is: ON


<h1> Monkey Patching</h1>

In the FrenchDeck example from chapter 1, the 'FrenchDeck' cannot be shuffled because the object does not support item assignment. The problem is that shuffle operates by swapping items inside the collection, and FrenchDeck only implements the immutable sequence protocol. Mutable sequences must also provide a __setitem__ method.

Because Python is dynamic, we can fix this at runtime.

First, create the FrenchDeck class

In [5]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]
   
    def __len__(self):
        return len(self._cards) 
    
    def __getitem__(self, position): 
        return self._cards[position]

As explained above, attempting to shuffle the deck will produce a TypeError

In [6]:
>>> from random import shuffle
>>> deck = FrenchDeck()
>>> shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

<h3>Solution: Monkey patch it</h3>

In [7]:
>>> def set_card(deck, position, card):
        deck._cards[position] = card
>>> FrenchDeck.__setitem__ = set_card
>>> shuffle(deck)
>>> deck[:5]
[Card(rank='3', suit='hearts'), Card(rank='4', suit='diamonds'), Card(rank='4', suit='clubs'), Card(rank='7', suit='hearts'), Card(rank='9', suit='spades')]

[Card(rank='3', suit='hearts'), Card(rank='4', suit='diamonds'), Card(rank='4', suit='clubs'), Card(rank='7', suit='hearts'), Card(rank='9', suit='spades')]

<h3>Or we could subclass the collections.MutableSequence ABC</h3>

In [14]:
class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, value):
        self._cards.insert(position, value)

<h1>Creating an abstract base class (ABC)</h1>

This example creates an ABC to support user-provided nonrepeating random-picking classes.

It is named [*Tombola*](http://www.oxforddictionaries.com/us/definition/american_english/tombola), after the Italian name of bingo and the tumbling container that mixes the numbers. 

This ABC will have 
* two abstract methods -- .load() and .pick()
    
    * We can’t know how concrete subclasses will store the items, but we can build the inspect result by emptying the Tombola with successive calls to .pick() and then use .load() to put everything back.
    
* two concrete methods -- .loaded() and .inspect(). 
     
     * It's OK to provide concrete methods in ABCs, as long as they only depend on other methods in the interface

In [9]:
import abc

In [24]:
class Tombola(abc.ABC):   #subclass abc.ABC to define an ABC in Python 3.4+
 
    @abc.abstractmethod   #An abstract method is marked with the @abstractmethod decorator. 
                          #Often its body is empty except for a docstring.
    def load(self, iterable):     
        """Add items from an iterable."""
 
    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it. This method should raise `LookupError` when the instance is empty."""
    
    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise.""" 
        return bool(self.inspect())

    def inspect(self): 
        """Return a sorted tuple with the items currently inside.""" 
        items = [] 
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

To witness the interface checking per‐ formed by an ABC, let’s try to fool Tombola with a defective implementation:

In [16]:
from tombola import Tombola
class Fake(Tombola):
    def pick(self): 
        return 13

In [73]:
Fake

__main__.Fake

In [22]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

We should see "TypeError: Can't instantiate abstract class Fake with abstract methods load"

Fake is considered abstract because it failed to implement load, one of the abstract methods declared in the Tombola ABC

<h1>Given the Tombola ABC, we’ll now develop two concrete subclasses that satisfy its interface: BingoCage, and LotteryBlower</h1>

<h2>bingo.py: BingoCage is a concrete subclass of Tombola</h2>

In [25]:
import random
from tombola import Tombola

In [28]:
class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom() 
        self._items = []
        self.load(items)
        
    def load(self, items): 
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

we can be lazy and just inherit the suboptimal concrete methods from the Tombola ABC, ie. inspect() and loaded()

<h2>lotto.py: LotteryBlower is a concrete subclass that overrides the inspect and loaded methods from Tombola</h2>

In [ ]:
import random
from tombola import Tombola

In [33]:
class LotteryBlower(Tombola): 
    
    def __init__(self, iterable):
        self._balls = list(iterable)   #flexible because the iterable argument may be any iterable type 
                                       #but are stored in as list
        
    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingoCage') #catch ValueError and throw LookupError instead 
                                                           #to be compatible with Tombola
        return self._balls.pop(position)

    def loaded(self):    #Override loaded to avoid calling inspect
        return bool(self._balls)

    def inspect(self):   #Override inspect with one-liner.
        return tuple(sorted(self._balls))

<h1>Goose Typing</h1>

By calling a register method on the ABC, the registered class becomes a virtual subclass of the ABC, 
and will be recognized as such by functions like issubclass and isinstance, 
but it will not inherit any methods or attributes from the ABC

<h3>tombolist.py: class TomboList is a virtual subclass of Tombola</h3>

In [34]:
from random import randrange
from tombola import Tombola

In [37]:
@Tombola.register 
class TomboList(list):
    def pick(self): 
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
        load = list.extend

    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))

In [38]:
Tombola.register(TomboList)

__main__.TomboList

Note:  If you’re using Python 3.3 or earlier, you can’t use .register as a class decorator. You must use standard call syntax.

<h3>Test the virtual subclass</h3>

In [65]:
>>> from tombola import Tombola 
>>> from tombolist import TomboList
>>> issubclass(TomboList, Tombola) 
True
>>> t = TomboList(range(100))
>>> isinstance(t, Tombola)
True

True

<h3>Check the MRO (Method Resolution Order)</h3>

In [67]:
TomboList.__mro__

(tombolist.TomboList, list, object)

Tombola is not in Tombolist.__mro__, so Tombolist does not inherit any methods from Tombola

<h2>Run tombola_runner.py to test all Tombola subclasses</h2>

First, reset the interpreter to remove all user-defined names from the namespace

In [94]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [95]:
%run tombola_runner.py

TumblingDrum     24 tests,  0 failed - OK
LotteryBlower    24 tests,  0 failed - OK
BingoCage        24 tests,  0 failed - OK
TomboList        24 tests,  0 failed - OK


<h2>A class can be recognized as a virtual subclass of an ABC even without registration if the ABC implements a special class method named __subclasshook__ -- but this should rarely be used</h2>

In [98]:
>>> class Struggle:
        def __len__(self):
            return 23
        
>>> from collections import abc
>>> isinstance(Struggle(), abc.Sized) 
True
>>> issubclass(Struggle, abc.Sized) 
True

True